In [ ]:
pip install tqdm

In [1]:
import os
import sys

import pandas as pd
from os.path import isfile, dirname, join
from urllib.parse import urlparse

from os import listdir
from os.path import isfile, join, splitext, basename

import requests
from tqdm.notebook import tqdm


In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True )

Mounted at /content/drive


In [3]:
cd drive/

/content/drive


In [4]:
import os
import pandas as pd
from os.path import isfile, dirname, join

my_path = "MyDrive/raw_data/yoga_dataset_links"

from os import listdir
from os.path import isfile, join
onlyfiles = [f"{my_path}/{f}" for f in listdir(my_path) if isfile(join(my_path, f))]

In [5]:
for file in onlyfiles:
  print(file)

MyDrive/raw_data/yoga_dataset_links/Yogic_sleep_pose.txt
MyDrive/raw_data/yoga_dataset_links/Sitting pose 1 (normal).txt
MyDrive/raw_data/yoga_dataset_links/Split pose.txt
MyDrive/raw_data/yoga_dataset_links/Feathered_Peacock_Pose_or_Pincha_Mayurasana_.txt
MyDrive/raw_data/yoga_dataset_links/Supported_Shoulderstand_pose_or_Salamba_Sarvangasana_.txt
MyDrive/raw_data/yoga_dataset_links/Virasana_or_Vajrasana.txt
MyDrive/raw_data/yoga_dataset_links/viparita_virabhadrasana_or_reverse_warrior_pose.txt
MyDrive/raw_data/yoga_dataset_links/Extended_Revolved_Triangle_Pose_or_Utthita_Trikonasana_.txt
MyDrive/raw_data/yoga_dataset_links/Warrior_II_Pose_or_Virabhadrasana_II_.txt
MyDrive/raw_data/yoga_dataset_links/Standing_Split_pose_or_Urdhva_Prasarita_Eka_Padasana_.txt
MyDrive/raw_data/yoga_dataset_links/Wind_Relieving_pose_or_Pawanmuktasana.txt
MyDrive/raw_data/yoga_dataset_links/Revolved_Head-to-Knee_Pose_or_Parivrtta_Janu_Sirsasana_.txt
MyDrive/raw_data/yoga_dataset_links/Scale_Pose_or_Tolasan

In [6]:
domains = {}
for file in onlyfiles:
  if os.path.splitext(file)[1] != '.ini':
    print(f"inspecting {file}")
    df = pd.read_csv(file, delimiter = "\t", header=None)
    for link in df[1]:
        domain = urlparse(link).netloc.split('.')[-2]
        if not domain in domains.keys():
            domains[domain] = link

inspecting MyDrive/raw_data/yoga_dataset_links/Yogic_sleep_pose.txt
inspecting MyDrive/raw_data/yoga_dataset_links/Sitting pose 1 (normal).txt
inspecting MyDrive/raw_data/yoga_dataset_links/Split pose.txt
inspecting MyDrive/raw_data/yoga_dataset_links/Feathered_Peacock_Pose_or_Pincha_Mayurasana_.txt
inspecting MyDrive/raw_data/yoga_dataset_links/Supported_Shoulderstand_pose_or_Salamba_Sarvangasana_.txt
inspecting MyDrive/raw_data/yoga_dataset_links/Virasana_or_Vajrasana.txt
inspecting MyDrive/raw_data/yoga_dataset_links/viparita_virabhadrasana_or_reverse_warrior_pose.txt
inspecting MyDrive/raw_data/yoga_dataset_links/Extended_Revolved_Triangle_Pose_or_Utthita_Trikonasana_.txt
inspecting MyDrive/raw_data/yoga_dataset_links/Warrior_II_Pose_or_Virabhadrasana_II_.txt
inspecting MyDrive/raw_data/yoga_dataset_links/Standing_Split_pose_or_Urdhva_Prasarita_Eka_Padasana_.txt
inspecting MyDrive/raw_data/yoga_dataset_links/Wind_Relieving_pose_or_Pawanmuktasana.txt
inspecting MyDrive/raw_data/yoga

In [7]:
### Create directories

In [8]:
dataset_root_path = "MyDrive/raw_data/images-bis"
if not os.path.exists(dataset_root_path):
    os.makedirs(dataset_root_path)

for file in onlyfiles:
  file_no_path = basename(file)
  filename, fileextension = splitext(file_no_path)
  if fileextension != '.ini':
    dataset_pose_path = dataset_root_path + "/" + filename
    if not os.path.exists(dataset_pose_path):
      os.makedirs(dataset_pose_path)
    

In [9]:
### iterate in poses folder to retrieve images

In [ ]:
def check_file_in_csv(path_to_file, ref_file):
    with open(path_to_file, 'r') as read_obj:
      for line in read_obj:
          if ref_file in line:
            return True
    return False


In [ ]:
path_to_file_success = "MyDrive/raw_data/images-bis/success.csv"
path_to_file_failed =  "MyDrive/raw_data/images-bis/failure.csv"

count = 1
for file in onlyfiles:
  print(f"Collecting image dataset from {file} : {count}/{len(onlyfiles)}")
  file_no_path = basename(file)
  pose_class, fileextension = splitext(file_no_path)
  if fileextension != '.ini':
    
    df = pd.read_csv(file, delimiter = "\t", header=None)
    dataset_pose_path = dataset_root_path + "/" + pose_class
    img_count = 1

    for link in tqdm(df[1]):
      img = basename(link)
      
      to_file = dataset_pose_path + "/" + img
      try:
        req = requests.get(link,timeout=2)
        if req.status_code == 200:
          with open(to_file, 'wb') as f:
            f.write(req.content)
          with open(path_to_file_success, "a") as file_success_csv:
            file_success_csv.write(pose_class +', ' + link +"\n")
        else:
          with open(path_to_file_failed, "a") as file_failed_csv:
            file_failed_csv.write(pose_class +', ' + link +"\n")
      except:
        with open(path_to_file_failed, "a") as file_failed_csv:
            file_failed_csv.write(pose_class +', ' + link +"\n")

  count+=1


In [ ]:
### once collected, get distribution balance of pose classes (should be roughly equivalent)
### to be checked from success.csv